In [1]:
import os

In [2]:
%pwd

'c:\\Users\\ROOT\\Desktop\\mlops_projects\\data_science_project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\ROOT\\Desktop\\mlops_projects\\data_science_project'

In [5]:
%ls

 Volume in drive C has no label.
 Volume Serial Number is 3C50-BC77

 Directory of c:\Users\ROOT\Desktop\mlops_projects\data_science_project

05/20/2025  06:55 AM    <DIR>          .
05/20/2025  06:55 AM    <DIR>          ..
05/20/2025  06:33 AM    <DIR>          .github
05/20/2025  06:04 AM                 5 .gitignore
05/20/2025  06:33 AM    <DIR>          config
05/20/2025  06:36 AM                 0 DockerFile
05/20/2025  06:02 AM            35,823 LICENSE
05/20/2025  06:49 AM                95 main.py
05/20/2025  06:36 AM                 0 params.yaml
05/20/2025  08:06 AM               393 README.md
05/20/2025  06:11 AM               132 requirements.txt
05/20/2025  08:08 AM    <DIR>          research
05/20/2025  06:36 AM                 0 schema.yaml
05/20/2025  06:36 AM                 0 setup.py
05/20/2025  06:33 AM    <DIR>          src
05/20/2025  06:40 AM             1,456 template.py
05/20/2025  06:33 AM    <DIR>          templates
05/20/2025  06:08 AM    <DIR>          ven

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionconfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories

In [16]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path = CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH,
                 schema_file_path = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)


        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionconfig:
        config = self.config.data_ingetion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionconfig(
            root_dir = config.root_dir,
            source_url = config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [17]:
import os
import urllib.request as request
from src.datascience import logger
import zipfile

In [22]:
## Components-Data Ingestion
class DataIngestion:
    def __init__(self, config:DataIngestionconfig):
        self.config = config

    # Downloading the zip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)        

In [23]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-05-20 08:59:00,107: INFO: common: Yaml File: config\config.yaml loaded successfully]
[2025-05-20 08:59:00,110: INFO: common: Yaml File: params.yaml loaded successfully]
[2025-05-20 08:59:00,112: INFO: common: Yaml File: schema.yaml loaded successfully]
[2025-05-20 08:59:00,113: INFO: common: Created Directory at: artifacts]
[2025-05-20 08:59:00,115: INFO: common: Created Directory at: artifacts/data_ingestion]
[2025-05-20 08:59:01,180: INFO: 1618756832: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 3535:194D25:12C36F:30E0FA:682BEFF5
Accept-Ranges: bytes
Date: Tue, 20